In [ ]:
import pandas as pd
import numpy as np
import string

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score,GridSearchCV, RandomizedSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier


In [ ]:
from wordcloud import WordCloud
from nltk.corpus import stopwords
from textblob import Word, TextBlob
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from nltk.stem import WordNetLemmatizer
import spacy

import matplotlib.pyplot as plt
import seaborn as sns


/opt/conda/lib/python3.10/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 55)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# **Завантаження повного датасету та препроцесинг**

In [ ]:
import pandas as pd
from datasets import load_dataset

# Завантаження датасету
dataset = load_dataset('mavinsao/reddit-mental-illness-82')
df_tr = dataset['train'].to_pandas()
df_val = dataset['validation'].to_pandas()
df_ts = dataset['test'].to_pandas()
df = pd.concat([df_tr, df_val, df_ts], axis=0, ignore_index=True)

# Перетворення тексту на нижній регістр
df['text'] = df['text'].str.lower()

# Фільтрація рядків за наявністю ключових слів
keywords = [
    "think i have", "i think i have", "i think it might be", "i think i could have",
    "might have", "i might have", "might be", "feel like i have", "i feel like i have",
    "feels like i have", "self-diagnose", "self-diagnosed", "i've self-diagnosed",
    "unsure if i have", "i'm unsure if", "unsure if this is", "wonder if i have",
    "i wonder if i have", "wonder if it's", "symptoms of", "i have symptoms of",
    "experiencing symptoms of", "suspect i have", "i suspect i have", "i suspect it's",
    "probably have", "i probably have", "i think i probably have", "could be", "it could be",
    "seems like i have", "it seems like i have", "it seems like", "not diagnosed but",
    "i am not diagnosed but", "i haven't been diagnosed but"
]

df = df[~df['text'].str.contains('|'.join(keywords), case=False, na=False)]

# Розбиття тексту на заголовок і основний текст
def split_text(row):
    if ':' in row:
        parts = row.split(':', 1)
        return parts[0].strip(), parts[1].strip()
    return None, row.strip()

df[['title', 'main_text']] = df['text'].apply(split_text).apply(pd.Series)
print(df[['title', 'main_text', 'label']])


README.md:   0%|          | 0.00/755 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/24.2M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/3.03M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/3.04M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/42113 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5264 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5265 [00:00<?, ? examples/s]

                                                   title  \
0                              fantasizing about your fp   
1      this is a support subreddit for people with a ...   
2                        scared of my psychotic symptoms   
4      i feel like a sick animal that needs to be put...   
5      is constantly checking ocd subreddits a compul...   
...                                                  ...   
52637  i got diagnosed with pure o, and finding it di...   
52638                 where can i read up on propaganda?   
52639  need help "keeping thoughts straight," or, mai...   
52640  does anyone else feel like none of their exper...   
52641                         wish me luck at work today   

                                               main_text  label  
0      do you do it? what do you fantasize about? : w...      3  
1      it's not for posting how infuriating the uneve...      6  
2      i'm trying to keep reminding myself that what ...      2  
4      i had a 

In [ ]:
import pandas as pd
from nltk.corpus import stopwords
import spacy

# Функція очищення тексту
def clean_text(text):
    text = text.str.lower()
    text = text.str.replace(r'[^\w\s]', '', regex=True)
    text = text.str.replace("\n", '', regex=True)
    text = text.str.replace('\d', '', regex=True)
    text = text.str.replace(r'\[.*?\]', '', regex=True)
    text = text.str.replace(r'https?://\S+|www\.\S+', '', regex=True)
    text = text.str.replace(r'<.*?>+', '', regex=True)
    text = text.str.replace(r'\w*\d\w*', '', regex=True)
    return text

# Функція видалення стоп-слів
# custom_stopwords = {'ocd', 'anxiety', 'adhd', 'ptsd', 'bpd', 'depression', 'bipolar'}
custom_stopwords = {}
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    stop_words.update(custom_stopwords)
    text = text.apply(lambda x: " ".join(word for word in str(x).split() if word.lower() not in stop_words))
    return text

# Функція лематизації
nlp = spacy.load('en_core_web_sm')

def lemmatize_sentence(sentence):
    doc = nlp(sentence)
    return " ".join([token.lemma_ for token in doc])

# Застосування функцій до обох стовпців
def process_text_columns(df, columns):
    for col in columns:
        df[col] = clean_text(df[col])
        df[col] = remove_stopwords(df[col])
        delete = pd.Series(' '.join(df[col]).split()).value_counts()[-1000:]
        df[col] = df[col].apply(lambda x: " ".join(word for word in x.split() if word.lower() not in delete))
        df[col] = df[col].apply(lemmatize_sentence)
    return df

df = pd.DataFrame(df)

# Застосування функцій до стовпців 'title' і 'main_text'
df = process_text_columns(df, ['title', 'main_text'])

# Результат
print(df)


                                                    text  label  \
0      fantasizing about your fp: do you do it? what ...      3   
1      this is a support subreddit for people with a ...      6   
2      scared of my psychotic symptoms : i'm trying t...      2   
4      i feel like a sick animal that needs to be put...      4   
5      is constantly checking ocd subreddits a compul...      6   
...                                                  ...    ...   
52637  i got diagnosed with pure o, and finding it di...      6   
52638  where can i read up on propaganda? : i want to...      5   
52639  need help "keeping thoughts straight," or, mai...      0   
52640  does anyone else feel like none of their exper...      1   
52641  wish me luck at work today : oh god my anxiety...      1   

                                                   title  \
0                                           fantasize fp   
1                support subreddit people mental illness   
2              

In [ ]:
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split




# Об'єднання заголовку і основного тексту в один стовпець
df['text'] = df['title'].str.strip(':') + ' ' + df['main_text']
# df['text'] = df['main_text']
# Розподіл на X (тексти) та y (мітки)
X = df['text']
y = df['label']

# Ініціалізація RandomUnderSampler для балансування класів
undersampler = RandomUnderSampler(sampling_strategy='auto', random_state=42)

# Застосування undersampling
X_res, y_res = undersampler.fit_resample(X.values.reshape(-1, 1), y)
print("Розподіл класів після балансування:")
print(pd.Series(y_res).value_counts())
balanced_df = pd.DataFrame({
    'text': X_res.flatten(),
    'label': y_res
})

train_df, test_df = train_test_split(balanced_df, test_size=0.2, random_state=42)
print("\nРозподіл класів у тренувальному наборі:")
print(train_df['label'].value_counts())
print("\nРозподіл класів у тестовому наборі:")
print(test_df['label'].value_counts())


Розподіл класів після балансування:
label
0    4086
1    4086
2    4086
3    4086
4    4086
5    4086
6    4086
7    4086
Name: count, dtype: int64

Розподіл класів у тренувальному наборі:
label
5    3335
0    3296
1    3279
4    3264
6    3264
3    3260
2    3228
7    3224
Name: count, dtype: int64

Розподіл класів у тестовому наборі:
label
7    862
2    858
3    826
6    822
4    822
1    807
0    790
5    751
Name: count, dtype: int64


# **LogisticRegression**

# TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

# Трансформація тексту в TF-IDF вектори
tfidf_vectorizer = TfidfVectorizer()
X_train = tfidf_vectorizer.fit_transform(train_df['text'])
X_test = tfidf_vectorizer.transform(test_df['text'])

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_df['label'])
y_test = label_encoder.transform(test_df['label'])

# Логістична регресія
logistic_regression_classifier = LogisticRegression(max_iter=1000, random_state=42)
logistic_regression_classifier.fit(X_train, y_train)

y_test_pred = logistic_regression_classifier.predict(X_test)
print("Test Set Evaluation:")
print(f"Accuracy: {accuracy_score(y_test, y_test_pred)}")
print(classification_report(y_test, y_test_pred))

Test Set Evaluation:
Accuracy: 0.7872438054450902
              precision    recall  f1-score   support

           0       0.81      0.83      0.82       790
           1       0.76      0.74      0.75       807
           2       0.79      0.70      0.74       858
           3       0.73      0.69      0.71       826
           4       0.63      0.78      0.69       822
           5       0.88      0.98      0.93       751
           6       0.90      0.81      0.85       822
           7       0.83      0.79      0.81       862

    accuracy                           0.79      6538
   macro avg       0.79      0.79      0.79      6538
weighted avg       0.79      0.79      0.79      6538



# Word2Vec

In [ ]:
from gensim.models import Word2Vec
import numpy as np

word2vec_model = Word2Vec(sentences=[text.split() for text in train_df['text']], vector_size=100, window=5, min_count=1, workers=4)
def text_to_avg_vector(text, model):
    words = text.split()
    vectors = [model.wv[word] for word in words if word in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

X_train_w2v = np.array([text_to_avg_vector(text, word2vec_model) for text in train_df['text']])
X_test_w2v = np.array([text_to_avg_vector(text, word2vec_model) for text in test_df['text']])
logistic_regression_classifier_w2v = LogisticRegression(max_iter=1000, random_state=42)
logistic_regression_classifier_w2v.fit(X_train_w2v, y_train)

y_test_pred_w2v = logistic_regression_classifier_w2v.predict(X_test_w2v)
print("Word2Vec Evaluation:")
print(f"Accuracy: {accuracy_score(y_test, y_test_pred_w2v)}")
print(classification_report(y_test, y_test_pred_w2v))


Word2Vec Evaluation:
Accuracy: 0.7049556439278066
              precision    recall  f1-score   support

           0       0.68      0.74      0.70       790
           1       0.71      0.67      0.69       807
           2       0.67      0.59      0.63       858
           3       0.59      0.55      0.57       826
           4       0.56      0.64      0.59       822
           5       0.94      0.96      0.95       751
           6       0.83      0.76      0.79       822
           7       0.72      0.76      0.74       862

    accuracy                           0.70      6538
   macro avg       0.71      0.71      0.71      6538
weighted avg       0.71      0.70      0.70      6538



# FastText

In [ ]:
from gensim.models.fasttext import FastText

fasttext_model = FastText(sentences=[text.split() for text in train_df['text']], vector_size=100, window=5, min_count=1, workers=4)
X_train_fasttext = np.array([text_to_avg_vector(text, fasttext_model) for text in train_df['text']])
X_test_fasttext = np.array([text_to_avg_vector(text, fasttext_model) for text in test_df['text']])
logistic_regression_classifier_fasttext = LogisticRegression(max_iter=1000, random_state=42)
logistic_regression_classifier_fasttext.fit(X_train_fasttext, y_train)
y_test_pred_fasttext = logistic_regression_classifier_fasttext.predict(X_test_fasttext)

print("FastText Evaluation:")
print(f"Accuracy: {accuracy_score(y_test, y_test_pred_fasttext)}")
print(classification_report(y_test, y_test_pred_fasttext))


FastText Evaluation:
Accuracy: 0.7018966044661976
              precision    recall  f1-score   support

           0       0.66      0.77      0.71       790
           1       0.68      0.65      0.67       807
           2       0.66      0.55      0.60       858
           3       0.61      0.58      0.59       826
           4       0.56      0.64      0.60       822
           5       0.92      0.95      0.93       751
           6       0.83      0.78      0.80       822
           7       0.72      0.73      0.72       862

    accuracy                           0.70      6538
   macro avg       0.71      0.71      0.70      6538
weighted avg       0.70      0.70      0.70      6538



# Bag of Words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Трансформація тексту в Bag of Words вектори
bow_vectorizer = CountVectorizer()
X_train_bow = bow_vectorizer.fit_transform(train_df['text'])
X_test_bow = bow_vectorizer.transform(test_df['text'])
logistic_regression_classifier_bow = LogisticRegression(max_iter=1000, random_state=42)
logistic_regression_classifier_bow.fit(X_train_bow, y_train)

y_test_pred_bow = logistic_regression_classifier_bow.predict(X_test_bow)
print("Bag of Words Evaluation:")
print(f"Accuracy: {accuracy_score(y_test, y_test_pred_bow)}")
print(classification_report(y_test, y_test_pred_bow))


Bag of Words Evaluation:
Accuracy: 0.7577240746405629
              precision    recall  f1-score   support

           0       0.82      0.78      0.80       790
           1       0.70      0.70      0.70       807
           2       0.71      0.70      0.70       858
           3       0.69      0.67      0.68       826
           4       0.60      0.71      0.65       822
           5       0.94      0.94      0.94       751
           6       0.84      0.81      0.82       822
           7       0.81      0.77      0.79       862

    accuracy                           0.76      6538
   macro avg       0.76      0.76      0.76      6538
weighted avg       0.76      0.76      0.76      6538



# **XGBClassifier**

# TF-IDF

In [ ]:
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

tfidf_vectorizer = TfidfVectorizer()
X_train = tfidf_vectorizer.fit_transform(train_df['text'])
X_test = tfidf_vectorizer.transform(test_df['text'])
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_df['label'])
y_test = label_encoder.transform(test_df['label'])

# XGBoost
xgb_classifier = XGBClassifier(
    max_depth=3,
    n_estimators=300,
    learning_rate=0.1,
    random_state=42,
    use_label_encoder=False
)
xgb_classifier.fit(X_train, y_train)
y_test_pred = xgb_classifier.predict(X_test)

print("Test Set Evaluation:")
print(f"Accuracy: {accuracy_score(y_test, y_test_pred)}")
print(classification_report(y_test, y_test_pred))


Test Set Evaluation:
Accuracy: 0.7837259100642399
              precision    recall  f1-score   support

           0       0.85      0.80      0.82       790
           1       0.77      0.76      0.77       807
           2       0.84      0.69      0.76       858
           3       0.73      0.68      0.70       826
           4       0.59      0.78      0.67       822
           5       0.81      0.97      0.88       751
           6       0.89      0.82      0.85       822
           7       0.87      0.79      0.83       862

    accuracy                           0.78      6538
   macro avg       0.79      0.79      0.79      6538
weighted avg       0.79      0.78      0.78      6538



# Bag of Words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from xgboost import XGBClassifier

# Трансформація тексту в BoW
bow_vectorizer = CountVectorizer()
X_train_bow = bow_vectorizer.fit_transform(train_df['text'])
X_test_bow = bow_vectorizer.transform(test_df['text'])
xgb_classifier_bow = XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.1, random_state=42, use_label_encoder=False)
xgb_classifier_bow.fit(X_train_bow, y_train)
y_test_pred_bow = xgb_classifier_bow.predict(X_test_bow)
print("Bag of Words with XGBoost Evaluation:")
print(f"Accuracy: {accuracy_score(y_test, y_test_pred_bow)}")
print(classification_report(y_test, y_test_pred_bow))


Bag of Words with XGBoost Evaluation:
Accuracy: 0.7812786784949526
              precision    recall  f1-score   support

           0       0.85      0.78      0.82       790
           1       0.78      0.75      0.77       807
           2       0.86      0.68      0.76       858
           3       0.73      0.69      0.71       826
           4       0.60      0.77      0.68       822
           5       0.75      0.98      0.85       751
           6       0.90      0.82      0.86       822
           7       0.86      0.80      0.83       862

    accuracy                           0.78      6538
   macro avg       0.79      0.78      0.78      6538
weighted avg       0.79      0.78      0.78      6538



# Word2Vec

In [ ]:
from gensim.models import Word2Vec
import numpy as np

# Тренування Word2Vec
word2vec_model = Word2Vec(sentences=[text.split() for text in train_df['text']], vector_size=100, window=5, min_count=1, workers=4)

def text_to_avg_vector(text, model):
    words = text.split()
    vectors = [model.wv[word] for word in words if word in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

X_train_w2v = np.array([text_to_avg_vector(text, word2vec_model) for text in train_df['text']])
X_test_w2v = np.array([text_to_avg_vector(text, word2vec_model) for text in test_df['text']])
xgb_classifier_w2v = XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.1, random_state=42, use_label_encoder=False)
xgb_classifier_w2v.fit(X_train_w2v, y_train)
y_test_pred_w2v = xgb_classifier_w2v.predict(X_test_w2v)
print("Word2Vec with XGBoost Evaluation:")
print(f"Accuracy: {accuracy_score(y_test, y_test_pred_w2v)}")
print(classification_report(y_test, y_test_pred_w2v))


Word2Vec with XGBoost Evaluation:
Accuracy: 0.6495870296726828
              precision    recall  f1-score   support

           0       0.63      0.69      0.66       790
           1       0.62      0.60      0.61       807
           2       0.58      0.51      0.54       858
           3       0.52      0.49      0.51       826
           4       0.51      0.59      0.55       822
           5       0.93      0.95      0.94       751
           6       0.74      0.69      0.72       822
           7       0.69      0.69      0.69       862

    accuracy                           0.65      6538
   macro avg       0.65      0.65      0.65      6538
weighted avg       0.65      0.65      0.65      6538



# FastText

In [ ]:
from gensim.models.fasttext import FastText

# Тренування FastText
fasttext_model = FastText(sentences=[text.split() for text in train_df['text']], vector_size=100, window=5, min_count=1, workers=4)
X_train_fasttext = np.array([text_to_avg_vector(text, fasttext_model) for text in train_df['text']])
X_test_fasttext = np.array([text_to_avg_vector(text, fasttext_model) for text in test_df['text']])
xgb_classifier_fasttext = XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.1, random_state=42, use_label_encoder=False)
xgb_classifier_fasttext.fit(X_train_fasttext, y_train)
y_test_pred_fasttext = xgb_classifier_fasttext.predict(X_test_fasttext)

print("FastText with XGBoost Evaluation:")
print(f"Accuracy: {accuracy_score(y_test, y_test_pred_fasttext)}")
print(classification_report(y_test, y_test_pred_fasttext))


FastText with XGBoost Evaluation:
Accuracy: 0.6085959008871215
              precision    recall  f1-score   support

           0       0.59      0.67      0.62       790
           1       0.57      0.54      0.56       807
           2       0.52      0.46      0.49       858
           3       0.46      0.46      0.46       826
           4       0.48      0.55      0.51       822
           5       0.91      0.93      0.92       751
           6       0.74      0.68      0.71       822
           7       0.65      0.62      0.63       862

    accuracy                           0.61      6538
   macro avg       0.61      0.61      0.61      6538
weighted avg       0.61      0.61      0.61      6538



# **LSTM**

# keras Tokenizer

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import optuna

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df['text'])
X_train = tokenizer.texts_to_sequences(train_df['text'])
X_test = tokenizer.texts_to_sequences(test_df['text'])


max_len = max(len(x) for x in X_train + X_test)

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_df['label'])
y_test = label_encoder.transform(test_df['label'])

class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = torch.tensor(texts, dtype=torch.long)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

train_dataset = TextDataset(X_train, y_train)
test_dataset = TextDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

class AttentionLayer(nn.Module):
    def __init__(self, lstm_units):
        super(AttentionLayer, self).__init__()
        self.attention = nn.Linear(lstm_units * 2, 1)

    def forward(self, x):
        scores = self.attention(x)
        weights = torch.softmax(scores, dim=1)
        context = torch.sum(weights * x, dim=1)
        return context

class ThreeLayerLSTMWithAttention(nn.Module):
    def __init__(self, vocab_size, embed_dim, lstm_units, output_dim, dropout_rate):
        super(ThreeLayerLSTMWithAttention, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm1 = nn.LSTM(embed_dim, lstm_units, batch_first=True, dropout=dropout_rate, bidirectional=True)
        self.lstm2 = nn.LSTM(lstm_units * 2, lstm_units, batch_first=True, dropout=dropout_rate, bidirectional=True)
        self.lstm3 = nn.LSTM(lstm_units * 2, lstm_units, batch_first=True, dropout=dropout_rate, bidirectional=True)
        self.attention = AttentionLayer(lstm_units)
        self.fc = nn.Linear(lstm_units * 2, output_dim)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x, _ = self.lstm3(x)
        x = self.attention(x)
        x = self.dropout(x)
        x = self.fc(x)
        return x

output_dim = len(np.unique(y_train))

# Optuna Objective Function
def objective(trial):
    # Пошук оптимальних гіперпараметрів
    embed_dim = trial.suggest_categorical('embed_dim', [50, 100, 200])
    lstm_units = trial.suggest_int('lstm_units', 64, 256, step=32)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5, step=0.1)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)

    # Ініціалізація моделі
    model = ThreeLayerLSTMWithAttention(len(tokenizer.word_index) + 1, embed_dim, lstm_units, output_dim, dropout_rate).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Тренування
    def train_model(model, train_loader, optimizer, criterion, device, epochs=10):
        model.train()
        for epoch in range(epochs):
            total_loss, correct = 0, 0
            for texts, labels in train_loader:
                texts, labels = texts.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(texts)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
                correct += (outputs.argmax(1) == labels).sum().item()
            accuracy = correct / len(train_loader.dataset)
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}, Accuracy: {accuracy:.4f}")

    train_model(model, train_loader, optimizer, criterion, device)

    # Оцінка моделі
    def evaluate_model(model, test_loader, device):
        model.eval()
        total_loss, correct = 0, 0
        all_preds, all_labels = [], []
        with torch.no_grad():
            for texts, labels in test_loader:
                texts, labels = texts.to(device), labels.to(device)
                outputs = model(texts)
                loss = criterion(outputs, labels)
                total_loss += loss.item()
                preds = outputs.argmax(1).cpu().numpy()
                correct += (outputs.argmax(1) == labels).sum().item()
                all_preds.extend(preds)
                all_labels.extend(labels.cpu().numpy())
        accuracy = correct / len(test_loader.dataset)
        print(f"Test Loss: {total_loss / len(test_loader):.4f}, Test Accuracy: {accuracy:.4f}")
        return accuracy

    test_acc = evaluate_model(model, test_loader, device)
    return test_acc

# Optuna Study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)
print("Best Parameters:", study.best_params)
print("Best Accuracy:", study.best_value)


Using device: cuda


[I 2025-01-25 23:52:38,354] A new study created in memory with name: no-name-5209f5ef-7ea0-4008-bf06-b11fee9e32fb


Epoch 1/10, Loss: 1.2855, Accuracy: 0.5353
Epoch 2/10, Loss: 0.7400, Accuracy: 0.7564
Epoch 3/10, Loss: 0.6098, Accuracy: 0.8010
Epoch 4/10, Loss: 0.5136, Accuracy: 0.8364
Epoch 5/10, Loss: 0.4645, Accuracy: 0.8509
Epoch 6/10, Loss: 0.4647, Accuracy: 0.8512
Epoch 7/10, Loss: 0.5136, Accuracy: 0.8341
Epoch 8/10, Loss: 0.6071, Accuracy: 0.8021
Epoch 9/10, Loss: 0.5787, Accuracy: 0.8096
Epoch 10/10, Loss: 0.5249, Accuracy: 0.8283


[I 2025-01-26 00:27:01,018] Trial 0 finished with value: 0.7431936371979199 and parameters: {'embed_dim': 50, 'lstm_units': 192, 'dropout_rate': 0.2, 'learning_rate': 0.008646939722585803}. Best is trial 0 with value: 0.7431936371979199.


Test Loss: 0.8098, Test Accuracy: 0.7432
Epoch 1/10, Loss: 1.8305, Accuracy: 0.2653
Epoch 2/10, Loss: 1.0062, Accuracy: 0.6556
Epoch 3/10, Loss: 0.7310, Accuracy: 0.7619
Epoch 4/10, Loss: 0.6007, Accuracy: 0.8062
Epoch 5/10, Loss: 0.5048, Accuracy: 0.8403
Epoch 6/10, Loss: 0.4203, Accuracy: 0.8686
Epoch 7/10, Loss: 0.3432, Accuracy: 0.8950
Epoch 8/10, Loss: 0.2714, Accuracy: 0.9201
Epoch 9/10, Loss: 0.2195, Accuracy: 0.9368
Epoch 10/10, Loss: 0.1724, Accuracy: 0.9514


...

Test Loss: 1.0903, Test Accuracy: 0.7530
Epoch 1/10, Loss: 1.8522, Accuracy: 0.2622
Epoch 2/10, Loss: 1.0890, Accuracy: 0.6206
Epoch 3/10, Loss: 0.8163, Accuracy: 0.7283
Epoch 4/10, Loss: 0.7116, Accuracy: 0.7623
Epoch 5/10, Loss: 0.6728, Accuracy: 0.7804
Epoch 6/10, Loss: 0.5782, Accuracy: 0.8143
Epoch 7/10, Loss: 0.5227, Accuracy: 0.8324
Epoch 8/10, Loss: 0.4628, Accuracy: 0.8520
Epoch 9/10, Loss: 0.4099, Accuracy: 0.8694
Epoch 10/10, Loss: 0.3526, Accuracy: 0.8894


[I 2025-01-26 01:28:37,460] Trial 7 finished with value: 0.7488528602018966 and parameters: {'embed_dim': 100, 'lstm_units': 224, 'dropout_rate': 0.5, 'learning_rate': 0.00025579738459250107}. Best is trial 3 with value: 0.7561945549097583.


Test Loss: 0.8773, Test Accuracy: 0.7489
Epoch 2/10, Loss: 1.0734, Accuracy: 0.6300
Epoch 3/10, Loss: 0.8328, Accuracy: 0.7206
Epoch 4/10, Loss: 0.7231, Accuracy: 0.7599
Epoch 5/10, Loss: 0.6486, Accuracy: 0.7850
Epoch 6/10, Loss: 0.5904, Accuracy: 0.8048
Epoch 7/10, Loss: 0.5233, Accuracy: 0.8269
Epoch 8/10, Loss: 0.4576, Accuracy: 0.8520
Epoch 10/10, Loss: 0.3165, Accuracy: 0.8990


[I 2025-01-26 01:44:10,427] Trial 8 finished with value: 0.7488528602018966 and parameters: {'embed_dim': 50, 'lstm_units': 224, 'dropout_rate': 0.1, 'learning_rate': 0.00045178265641822226}. Best is trial 3 with value: 0.7561945549097583.


Test Loss: 0.9129, Test Accuracy: 0.7489


# Word2Vec

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec
import optuna

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
sentences = [text.split() for text in train_df['text']]  # Токенізація текстів

# Тренування Word2Vec
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Перетворення текстів у послідовності
word_index = {word: i + 1 for i, word in enumerate(word2vec_model.wv.index_to_key)}
vocab_size = len(word_index) + 1

def text_to_sequences(texts, word_index):
    sequences = []
    for text in texts:
        sequences.append([word_index.get(word, 0) for word in text.split()])
    return sequences

X_train = text_to_sequences(train_df['text'], word_index)
X_test = text_to_sequences(test_df['text'], word_index)

max_len = max(len(x) for x in X_train + X_test)
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_df['label'])
y_test = label_encoder.transform(test_df['label'])

class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = torch.tensor(texts, dtype=torch.long)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

train_dataset = TextDataset(X_train, y_train)
test_dataset = TextDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Створення векторів для слів
embedding_dim = word2vec_model.vector_size
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in word_index.items():
    embedding_matrix[i] = word2vec_model.wv[word]

class AttentionLayer(nn.Module):
    def __init__(self, lstm_units):
        super(AttentionLayer, self).__init__()
        self.attention = nn.Linear(lstm_units * 2, 1)

    def forward(self, x):
        scores = self.attention(x)
        weights = torch.softmax(scores, dim=1)
        context = torch.sum(weights * x, dim=1)
        return context

class ThreeLayerLSTMWithAttention(nn.Module):
    def __init__(self, embedding_matrix, lstm_units, output_dim, dropout_rate):
        super(ThreeLayerLSTMWithAttention, self).__init__()
        vocab_size, embed_dim = embedding_matrix.shape
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data.copy_(torch.tensor(embedding_matrix))
        self.embedding.weight.requires_grad = False
        self.lstm1 = nn.LSTM(embed_dim, lstm_units, batch_first=True, dropout=dropout_rate, bidirectional=True)
        self.lstm2 = nn.LSTM(lstm_units * 2, lstm_units, batch_first=True, dropout=dropout_rate, bidirectional=True)
        self.lstm3 = nn.LSTM(lstm_units * 2, lstm_units, batch_first=True, dropout=dropout_rate, bidirectional=True)
        self.attention = AttentionLayer(lstm_units)
        self.fc = nn.Linear(lstm_units * 2, output_dim)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x, _ = self.lstm3(x)
        x = self.attention(x)
        x = self.dropout(x)
        x = self.fc(x)
        return x

output_dim = len(np.unique(y_train))

# Optuna Objective Function
def objective(trial):
    lstm_units = trial.suggest_int('lstm_units', 64, 256, step=32)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5, step=0.1)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)

    # Ініціалізація моделі
    model = ThreeLayerLSTMWithAttention(embedding_matrix, lstm_units, output_dim, dropout_rate).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Тренування
    def train_model(model, train_loader, optimizer, criterion, device, epochs=15):
        model.train()
        for epoch in range(epochs):
            total_loss, correct = 0, 0
            for texts, labels in train_loader:
                texts, labels = texts.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(texts)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
                correct += (outputs.argmax(1) == labels).sum().item()
            accuracy = correct / len(train_loader.dataset)
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}, Accuracy: {accuracy:.4f}")

    train_model(model, train_loader, optimizer, criterion, device)

    # Оцінка моделі
    def evaluate_model(model, test_loader, device):
        model.eval()
        total_loss, correct = 0, 0
        with torch.no_grad():
            for texts, labels in test_loader:
                texts, labels = texts.to(device), labels.to(device)
                outputs = model(texts)
                loss = criterion(outputs, labels)
                total_loss += loss.item()
                correct += (outputs.argmax(1) == labels).sum().item()
        accuracy = correct / len(test_loader.dataset)
        print(f"Test Loss: {total_loss / len(test_loader):.4f}, Test Accuracy: {accuracy:.4f}")
        return accuracy

    test_acc = evaluate_model(model, test_loader, device)
    return test_acc

# Optuna Study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)
print("Best Parameters:", study.best_params)
print("Best Accuracy:", study.best_value)


Using device: cuda


[I 2025-01-26 11:04:11,649] A new study created in memory with name: no-name-0c0f6d95-93d5-43a1-a845-8f5a6350a222


Epoch 1/15, Loss: 1.0784, Accuracy: 0.6177
Epoch 2/15, Loss: 0.7106, Accuracy: 0.7724
Epoch 3/15, Loss: 0.6416, Accuracy: 0.7934
Epoch 4/15, Loss: 0.5935, Accuracy: 0.8088
Epoch 5/15, Loss: 0.5486, Accuracy: 0.8230
Epoch 6/15, Loss: 0.5048, Accuracy: 0.8384
Epoch 7/15, Loss: 0.4456, Accuracy: 0.8568
Epoch 8/15, Loss: 0.3822, Accuracy: 0.8766
Epoch 9/15, Loss: 0.3231, Accuracy: 0.8963
Epoch 10/15, Loss: 0.2578, Accuracy: 0.9168
Epoch 11/15, Loss: 0.2029, Accuracy: 0.9355
Epoch 12/15, Loss: 0.1627, Accuracy: 0.9491
Epoch 13/15, Loss: 0.1289, Accuracy: 0.9579
Epoch 14/15, Loss: 0.1158, Accuracy: 0.9638
Epoch 15/15, Loss: 0.0888, Accuracy: 0.9722


[I 2025-01-26 11:13:34,782] Trial 0 finished with value: 0.7809727745487917 and parameters: {'lstm_units': 128, 'dropout_rate': 0.4, 'learning_rate': 0.0017795750765174576}. Best is trial 0 with value: 0.7809727745487917.


Test Loss: 1.1289, Test Accuracy: 0.7810
Epoch 1/15, Loss: 1.3820, Accuracy: 0.4649
Epoch 2/15, Loss: 0.7477, Accuracy: 0.7610
Epoch 3/15, Loss: 0.6484, Accuracy: 0.7928
Epoch 4/15, Loss: 0.6206, Accuracy: 0.7985
Epoch 5/15, Loss: 0.5796, Accuracy: 0.8126
Epoch 6/15, Loss: 0.5417, Accuracy: 0.8246
Epoch 7/15, Loss: 0.5169, Accuracy: 0.8329
Epoch 8/15, Loss: 0.4884, Accuracy: 0.8407
Epoch 9/15, Loss: 0.4601, Accuracy: 0.8507
Epoch 10/15, Loss: 0.4357, Accuracy: 0.8574
Epoch 11/15, Loss: 0.4066, Accuracy: 0.8668
Epoch 12/15, Loss: 0.3811, Accuracy: 0.8742
Epoch 13/15, Loss: 0.3674, Accuracy: 0.8801
Epoch 14/15, Loss: 0.4518, Accuracy: 0.8513
Epoch 15/15, Loss: 0.4075, Accuracy: 0.8650


...

Test Loss: 0.8992, Test Accuracy: 0.7634
Epoch 1/15, Loss: 1.1481, Accuracy: 0.5874
Epoch 2/15, Loss: 0.7259, Accuracy: 0.7636
Epoch 6/15, Loss: 0.5447, Accuracy: 0.8228
Epoch 7/15, Loss: 0.5056, Accuracy: 0.8337
Epoch 8/15, Loss: 0.4637, Accuracy: 0.8453
Epoch 9/15, Loss: 0.4183, Accuracy: 0.8605
Epoch 10/15, Loss: 0.3627, Accuracy: 0.8785
Epoch 11/15, Loss: 0.3116, Accuracy: 0.8961
Epoch 12/15, Loss: 0.2601, Accuracy: 0.9139
Epoch 13/15, Loss: 0.2093, Accuracy: 0.9311
Epoch 14/15, Loss: 0.1769, Accuracy: 0.9418
Epoch 15/15, Loss: 0.1357, Accuracy: 0.9552


[I 2025-01-26 13:43:23,536] Trial 5 finished with value: 0.7739369837870909 and parameters: {'lstm_units': 128, 'dropout_rate': 0.30000000000000004, 'learning_rate': 0.0008910405638158334}. Best is trial 1 with value: 0.784796573875803.


Test Loss: 1.0755, Test Accuracy: 0.7739
Epoch 1/15, Loss: 1.2975, Accuracy: 0.5250
Epoch 2/15, Loss: 0.7727, Accuracy: 0.7472
Epoch 3/15, Loss: 0.6933, Accuracy: 0.7743
Epoch 4/15, Loss: 0.6546, Accuracy: 0.7884
Epoch 5/15, Loss: 0.6172, Accuracy: 0.7998
Epoch 6/15, Loss: 0.5913, Accuracy: 0.8079
Epoch 7/15, Loss: 0.5680, Accuracy: 0.8153
Epoch 8/15, Loss: 0.5358, Accuracy: 0.8262
Epoch 9/15, Loss: 0.5128, Accuracy: 0.8343
Epoch 10/15, Loss: 0.4812, Accuracy: 0.8445
Epoch 11/15, Loss: 0.4558, Accuracy: 0.8530
Epoch 12/15, Loss: 0.4339, Accuracy: 0.8602
Epoch 13/15, Loss: 0.3996, Accuracy: 0.8709
Epoch 14/15, Loss: 0.3675, Accuracy: 0.8823
Epoch 15/15, Loss: 0.3393, Accuracy: 0.8889


[I 2025-01-26 13:46:01,724] Trial 6 finished with value: 0.7705720403793209 and parameters: {'lstm_units': 64, 'dropout_rate': 0.2, 'learning_rate': 0.0006156299620968943}. Best is trial 1 with value: 0.784796573875803.


Test Loss: 0.7811, Test Accuracy: 0.7706
Epoch 1/15, Loss: 1.4664, Accuracy: 0.4427
Epoch 2/15, Loss: 0.8901, Accuracy: 0.7020
Epoch 3/15, Loss: 0.7771, Accuracy: 0.7407
Epoch 4/15, Loss: 0.7268, Accuracy: 0.7586
Epoch 5/15, Loss: 0.6900, Accuracy: 0.7720
Epoch 6/15, Loss: 0.6687, Accuracy: 0.7775
Epoch 7/15, Loss: 0.6409, Accuracy: 0.7875
Epoch 8/15, Loss: 0.6235, Accuracy: 0.7944
Epoch 9/15, Loss: 0.6010, Accuracy: 0.8013
Epoch 10/15, Loss: 0.5780, Accuracy: 0.8086
Epoch 11/15, Loss: 0.5600, Accuracy: 0.8138


# TF-IDF

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import optuna

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
train_texts = train_df['text'].values
test_texts = test_df['text'].values

# Обчислення TF-IDF
vectorizer = TfidfVectorizer(max_features=10000)
X_train = vectorizer.fit_transform(train_texts).toarray()
X_test = vectorizer.transform(test_texts).toarray()
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_df['label'])
y_test = label_encoder.transform(test_df['label'])

class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = torch.tensor(texts, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

train_dataset = TextDataset(X_train, y_train)
test_dataset = TextDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Модель без шару Embedding
class SimpleNNWithAttention(nn.Module):
    def __init__(self, input_dim, lstm_units, output_dim, dropout_rate):
        super(SimpleNNWithAttention, self).__init__()
        self.lstm1 = nn.LSTM(input_dim, lstm_units, batch_first=True, dropout=dropout_rate, bidirectional=True)
        self.lstm2 = nn.LSTM(lstm_units * 2, lstm_units, batch_first=True, dropout=dropout_rate, bidirectional=True)
        self.lstm3 = nn.LSTM(lstm_units * 2, lstm_units, batch_first=True, dropout=dropout_rate, bidirectional=True)
        self.attention = nn.Linear(lstm_units * 2, 1)
        self.fc = nn.Linear(lstm_units * 2, output_dim)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x, _ = self.lstm1(x.unsqueeze(1))
        x, _ = self.lstm2(x)
        x, _ = self.lstm3(x)
        scores = self.attention(x)
        weights = torch.softmax(scores, dim=1)
        context = torch.sum(weights * x, dim=1)
        x = self.dropout(context)
        x = self.fc(x)
        return x

input_dim = X_train.shape[1]
output_dim = len(np.unique(y_train))

# Optuna Objective Function
def objective(trial):
    lstm_units = trial.suggest_int('lstm_units', 64, 256, step=32)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5, step=0.1)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)

    # Ініціалізація моделі
    model = SimpleNNWithAttention(input_dim, lstm_units, output_dim, dropout_rate).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Тренування
    def train_model(model, train_loader, optimizer, criterion, device, epochs=15):
        model.train()
        for epoch in range(epochs):
            total_loss, correct = 0, 0
            for texts, labels in train_loader:
                texts, labels = texts.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(texts)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
                correct += (outputs.argmax(1) == labels).sum().item()
            accuracy = correct / len(train_loader.dataset)
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}, Accuracy: {accuracy:.4f}")

    train_model(model, train_loader, optimizer, criterion, device)

    # Оцінка моделі
    def evaluate_model(model, test_loader, device):
        model.eval()
        total_loss, correct = 0, 0
        with torch.no_grad():
            for texts, labels in test_loader:
                texts, labels = texts.to(device), labels.to(device)
                outputs = model(texts)
                loss = criterion(outputs, labels)
                total_loss += loss.item()
                correct += (outputs.argmax(1) == labels).sum().item()
        accuracy = correct / len(test_loader.dataset)
        print(f"Test Loss: {total_loss / len(test_loader):.4f}, Test Accuracy: {accuracy:.4f}")
        return accuracy

    test_acc = evaluate_model(model, test_loader, device)
    return test_acc

# Optuna Study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)
print("Best Parameters:", study.best_params)
print("Best Accuracy:", study.best_value)


Using device: cuda


[I 2025-01-26 21:09:17,719] A new study created in memory with name: no-name-146849a6-7aa9-4ae9-9f46-6e4ce1f08ddf


Epoch 1/15, Loss: 0.9189, Accuracy: 0.6883
Epoch 2/15, Loss: 0.4247, Accuracy: 0.8671
Epoch 3/15, Loss: 0.2441, Accuracy: 0.9250
Epoch 4/15, Loss: 0.1468, Accuracy: 0.9521
Epoch 5/15, Loss: 0.0908, Accuracy: 0.9695
Epoch 6/15, Loss: 0.0697, Accuracy: 0.9773
Epoch 7/15, Loss: 0.0536, Accuracy: 0.9845
Epoch 8/15, Loss: 0.0301, Accuracy: 0.9912
Epoch 9/15, Loss: 0.0181, Accuracy: 0.9940
Epoch 10/15, Loss: 0.0196, Accuracy: 0.9943
Epoch 11/15, Loss: 0.0171, Accuracy: 0.9948
Epoch 12/15, Loss: 0.0199, Accuracy: 0.9938
Epoch 13/15, Loss: 0.0148, Accuracy: 0.9953
Epoch 14/15, Loss: 0.0196, Accuracy: 0.9944
Epoch 15/15, Loss: 0.0155, Accuracy: 0.9956


[I 2025-01-26 21:10:47,595] Trial 0 finished with value: 0.7485469562557358 and parameters: {'lstm_units': 256, 'dropout_rate': 0.5, 'learning_rate': 0.005604117449570261}. Best is trial 0 with value: 0.7485469562557358.


Test Loss: 1.8516, Test Accuracy: 0.7485
Epoch 1/15, Loss: 1.6059, Accuracy: 0.3709
Epoch 2/15, Loss: 0.7328, Accuracy: 0.7504
Epoch 3/15, Loss: 0.5071, Accuracy: 0.8368
Epoch 4/15, Loss: 0.3746, Accuracy: 0.8842
Epoch 5/15, Loss: 0.2768, Accuracy: 0.9172
Epoch 6/15, Loss: 0.2027, Accuracy: 0.9424
Epoch 7/15, Loss: 0.1477, Accuracy: 0.9608
Epoch 8/15, Loss: 0.1042, Accuracy: 0.9752
Epoch 9/15, Loss: 0.0704, Accuracy: 0.9842
Epoch 10/15, Loss: 0.0489, Accuracy: 0.9901
Epoch 11/15, Loss: 0.0342, Accuracy: 0.9935
Epoch 12/15, Loss: 0.0242, Accuracy: 0.9961
Epoch 13/15, Loss: 0.0176, Accuracy: 0.9972
Epoch 14/15, Loss: 0.0134, Accuracy: 0.9977
Epoch 15/15, Loss: 0.0102, Accuracy: 0.9984


...

Test Loss: 2.0541, Test Accuracy: 0.7400
Epoch 1/15, Loss: 0.9235, Accuracy: 0.6837
Epoch 2/15, Loss: 0.4293, Accuracy: 0.8672
Epoch 3/15, Loss: 0.2530, Accuracy: 0.9224
Epoch 4/15, Loss: 0.1585, Accuracy: 0.9507
Epoch 5/15, Loss: 0.1102, Accuracy: 0.9642
Epoch 6/15, Loss: 0.0715, Accuracy: 0.9776
Epoch 7/15, Loss: 0.0514, Accuracy: 0.9845
Epoch 8/15, Loss: 0.0344, Accuracy: 0.9893
Epoch 9/15, Loss: 0.0288, Accuracy: 0.9907
Epoch 10/15, Loss: 0.0248, Accuracy: 0.9925
Epoch 11/15, Loss: 0.0199, Accuracy: 0.9941
Epoch 12/15, Loss: 0.0113, Accuracy: 0.9961
Epoch 13/15, Loss: 0.0098, Accuracy: 0.9971
Epoch 14/15, Loss: 0.0190, Accuracy: 0.9948
Epoch 15/15, Loss: 0.0207, Accuracy: 0.9939


[I 2025-01-26 22:07:41,988] Trial 48 finished with value: 0.7372285102477822 and parameters: {'lstm_units': 192, 'dropout_rate': 0.5, 'learning_rate': 0.004782640484733326}. Best is trial 42 with value: 0.7502294279596207.


Test Loss: 1.9186, Test Accuracy: 0.7372
Epoch 1/15, Loss: 0.9536, Accuracy: 0.6714
Epoch 2/15, Loss: 0.4455, Accuracy: 0.8642
Epoch 3/15, Loss: 0.2495, Accuracy: 0.9262
Epoch 4/15, Loss: 0.1334, Accuracy: 0.9612
Epoch 5/15, Loss: 0.0785, Accuracy: 0.9778
Epoch 6/15, Loss: 0.0471, Accuracy: 0.9863
Epoch 7/15, Loss: 0.0392, Accuracy: 0.9883
Epoch 8/15, Loss: 0.0326, Accuracy: 0.9905
Epoch 9/15, Loss: 0.0281, Accuracy: 0.9915
Epoch 10/15, Loss: 0.0220, Accuracy: 0.9937
Epoch 11/15, Loss: 0.0138, Accuracy: 0.9964
Epoch 12/15, Loss: 0.0117, Accuracy: 0.9970
Epoch 13/15, Loss: 0.0199, Accuracy: 0.9941
Epoch 14/15, Loss: 0.0188, Accuracy: 0.9950
Epoch 15/15, Loss: 0.0138, Accuracy: 0.9963


[I 2025-01-26 22:08:31,437] Trial 49 finished with value: 0.7338635668400122 and parameters: {'lstm_units': 64, 'dropout_rate': 0.5, 'learning_rate': 0.00872002137873422}. Best is trial 42 with value: 0.7502294279596207.


Test Loss: 1.9768, Test Accuracy: 0.7339
Best Parameters: {'lstm_units': 192, 'dropout_rate': 0.5, 'learning_rate': 0.008268713053920792}
Best Accuracy: 0.7502294279596207


# FastText

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import LabelEncoder
from gensim.models import FastText
import optuna

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
sentences = [text.split() for text in train_df['text']]

# Тренування FastText
fasttext_model = FastText(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Перетворення текстів у послідовності
word_index = {word: i + 1 for i, word in enumerate(fasttext_model.wv.index_to_key)}  # Індексація слів
vocab_size = len(word_index) + 1  # Розмір словника з урахуванням паддінгів

def text_to_sequences(texts, word_index):
    sequences = []
    for text in texts:
        sequences.append([word_index.get(word, 0) for word in text.split()])
    return sequences

X_train = text_to_sequences(train_df['text'], word_index)
X_test = text_to_sequences(test_df['text'], word_index)
max_len = max(len(x) for x in X_train + X_test)
X_train = torch.tensor([seq + [0] * (max_len - len(seq)) for seq in X_train], dtype=torch.long)
X_test = torch.tensor([seq + [0] * (max_len - len(seq)) for seq in X_test], dtype=torch.long)
label_encoder = LabelEncoder()
y_train = torch.tensor(label_encoder.fit_transform(train_df['label']), dtype=torch.long)
y_test = torch.tensor(label_encoder.transform(test_df['label']), dtype=torch.long)
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

train_dataset = TextDataset(X_train, y_train)
test_dataset = TextDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Створення векторів для слів
embedding_dim = fasttext_model.vector_size
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in word_index.items():
    embedding_matrix[i] = fasttext_model.wv[word]

class AttentionLayer(nn.Module):
    def __init__(self, lstm_units):
        super(AttentionLayer, self).__init__()
        self.attention = nn.Linear(lstm_units * 2, 1)

    def forward(self, x):
        scores = self.attention(x)
        weights = torch.softmax(scores, dim=1)
        context = torch.sum(weights * x, dim=1)
        return context

class ThreeLayerLSTMWithAttention(nn.Module):
    def __init__(self, embedding_matrix, lstm_units, output_dim, dropout_rate):
        super(ThreeLayerLSTMWithAttention, self).__init__()
        vocab_size, embed_dim = embedding_matrix.shape
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data.copy_(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
        self.lstm1 = nn.LSTM(embed_dim, lstm_units, batch_first=True, dropout=dropout_rate, bidirectional=True)
        self.lstm2 = nn.LSTM(lstm_units * 2, lstm_units, batch_first=True, dropout=dropout_rate, bidirectional=True)
        self.lstm3 = nn.LSTM(lstm_units * 2, lstm_units, batch_first=True, dropout=dropout_rate, bidirectional=True)
        self.attention = AttentionLayer(lstm_units)
        self.fc = nn.Linear(lstm_units * 2, output_dim)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x, _ = self.lstm3(x)
        x = self.attention(x)
        x = self.dropout(x)
        x = self.fc(x)
        return x
output_dim = len(torch.unique(y_train))

# Optuna Objective Function
def objective(trial):
    lstm_units = trial.suggest_int('lstm_units', 64, 256, step=32)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5, step=0.1)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)

    # Ініціалізація моделі
    model = ThreeLayerLSTMWithAttention(embedding_matrix, lstm_units, output_dim, dropout_rate).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Тренування
    def train_model(model, train_loader, optimizer, criterion, device, epochs=15):
        model.train()
        for epoch in range(epochs):
            total_loss, correct = 0, 0
            for texts, labels in train_loader:
                texts, labels = texts.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(texts)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
                correct += (outputs.argmax(1) == labels).sum().item()
            accuracy = correct / len(train_loader.dataset)
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}, Accuracy: {accuracy:.4f}")

    train_model(model, train_loader, optimizer, criterion, device)

    # Оцінка моделі
    def evaluate_model(model, test_loader, device):
        model.eval()
        total_loss, correct = 0, 0
        with torch.no_grad():
            for texts, labels in test_loader:
                texts, labels = texts.to(device), labels.to(device)
                outputs = model(texts)
                loss = criterion(outputs, labels)
                total_loss += loss.item()
                correct += (outputs.argmax(1) == labels).sum().item()
        accuracy = correct / len(test_loader.dataset)
        print(f"Test Loss: {total_loss / len(test_loader):.4f}, Test Accuracy: {accuracy:.4f}")
        return accuracy

    test_acc = evaluate_model(model, test_loader, device)
    return test_acc

# Optuna Study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=25)
print("Best Parameters:", study.best_params)
print("Best Accuracy:", study.best_value)


Using device: cuda


[I 2025-01-26 22:51:05,962] A new study created in memory with name: no-name-5d86dc2b-0ca5-41fa-8af5-b09db8f259e1


Epoch 1/15, Loss: 1.1132, Accuracy: 0.6001
Epoch 2/15, Loss: 0.7176, Accuracy: 0.7718
Epoch 3/15, Loss: 0.6681, Accuracy: 0.7871
Epoch 4/15, Loss: 0.6270, Accuracy: 0.7997
Epoch 5/15, Loss: 0.6399, Accuracy: 0.7979
Epoch 6/15, Loss: 0.5882, Accuracy: 0.8107
Epoch 7/15, Loss: 0.5743, Accuracy: 0.8135
Epoch 8/15, Loss: 0.5698, Accuracy: 0.8151
Epoch 9/15, Loss: 0.5470, Accuracy: 0.8242
Epoch 10/15, Loss: 0.5410, Accuracy: 0.8228
Epoch 11/15, Loss: 0.5411, Accuracy: 0.8249
Epoch 12/15, Loss: 0.5405, Accuracy: 0.8240
Epoch 13/15, Loss: 0.5358, Accuracy: 0.8252
Epoch 14/15, Loss: 0.6290, Accuracy: 0.7965
Epoch 15/15, Loss: 0.5731, Accuracy: 0.8129


[I 2025-01-26 23:01:14,499] Trial 0 finished with value: 0.7936677883144693 and parameters: {'lstm_units': 128, 'dropout_rate': 0.4, 'learning_rate': 0.008120518921581686}. Best is trial 0 with value: 0.7936677883144693.


Test Loss: 0.6442, Test Accuracy: 0.7937
Epoch 1/15, Loss: 1.0596, Accuracy: 0.6263
Epoch 2/15, Loss: 0.6746, Accuracy: 0.7848
Epoch 3/15, Loss: 0.6103, Accuracy: 0.8050
Epoch 4/15, Loss: 0.5597, Accuracy: 0.8193
Epoch 5/15, Loss: 0.5084, Accuracy: 0.8346
Epoch 6/15, Loss: 0.4529, Accuracy: 0.8534
Epoch 7/15, Loss: 0.4083, Accuracy: 0.8655
Epoch 8/15, Loss: 0.3632, Accuracy: 0.8816
Epoch 9/15, Loss: 0.3166, Accuracy: 0.8976
Epoch 10/15, Loss: 0.2908, Accuracy: 0.9054
Epoch 11/15, Loss: 0.2565, Accuracy: 0.9170
Epoch 12/15, Loss: 0.2479, Accuracy: 0.9170
Epoch 13/15, Loss: 0.2050, Accuracy: 0.9333
Epoch 14/15, Loss: 0.1962, Accuracy: 0.9338
Epoch 15/15, Loss: 0.1914, Accuracy: 0.9370


...

Test Loss: 1.7396, Test Accuracy: 0.3538
Epoch 1/15, Loss: 1.7458, Accuracy: 0.2902
Epoch 2/15, Loss: 0.8466, Accuracy: 0.7215
Epoch 3/15, Loss: 0.6972, Accuracy: 0.7743
Epoch 4/15, Loss: 0.6527, Accuracy: 0.7891
Epoch 5/15, Loss: 0.6146, Accuracy: 0.8006
Epoch 6/15, Loss: 0.6087, Accuracy: 0.8028
Epoch 7/15, Loss: 0.5726, Accuracy: 0.8142
Epoch 8/15, Loss: 0.5933, Accuracy: 0.8070
Epoch 9/15, Loss: 0.5530, Accuracy: 0.8204
Epoch 10/15, Loss: 0.5161, Accuracy: 0.8323
Epoch 11/15, Loss: 0.4959, Accuracy: 0.8380
Epoch 12/15, Loss: 0.4791, Accuracy: 0.8421
Epoch 13/15, Loss: 0.4671, Accuracy: 0.8452
Epoch 14/15, Loss: 0.4532, Accuracy: 0.8529
Epoch 15/15, Loss: 0.4340, Accuracy: 0.8573


[I 2025-01-27 00:18:24,009] Trial 5 finished with value: 0.7921382685836648 and parameters: {'lstm_units': 224, 'dropout_rate': 0.1, 'learning_rate': 0.007397114131086175}. Best is trial 0 with value: 0.7936677883144693.


Test Loss: 0.6648, Test Accuracy: 0.7921
Epoch 1/15, Loss: 1.0456, Accuracy: 0.6368
Epoch 2/15, Loss: 0.7020, Accuracy: 0.7752
Epoch 3/15, Loss: 0.6480, Accuracy: 0.7942
Epoch 4/15, Loss: 0.5930, Accuracy: 0.8127
Epoch 5/15, Loss: 0.5460, Accuracy: 0.8261
Epoch 6/15, Loss: 0.4997, Accuracy: 0.8394
Epoch 7/15, Loss: 0.4519, Accuracy: 0.8553
Epoch 8/15, Loss: 0.4209, Accuracy: 0.8654
Epoch 9/15, Loss: 0.3772, Accuracy: 0.8797
Epoch 10/15, Loss: 0.3275, Accuracy: 0.8947
Epoch 11/15, Loss: 0.2970, Accuracy: 0.9063
Epoch 12/15, Loss: 0.2583, Accuracy: 0.9168
Epoch 13/15, Loss: 0.2345, Accuracy: 0.9238
Epoch 14/15, Loss: 0.2151, Accuracy: 0.9321
Epoch 15/15, Loss: 0.1846, Accuracy: 0.9398


[I 2025-01-27 00:23:58,756] Trial 6 finished with value: 0.7792903028449067 and parameters: {'lstm_units': 96, 'dropout_rate': 0.5, 'learning_rate': 0.003415265352030092}. Best is trial 0 with value: 0.7936677883144693.


Test Loss: 0.9166, Test Accuracy: 0.7793
Epoch 1/15, Loss: 1.3624, Accuracy: 0.4901
Epoch 2/15, Loss: 0.8202, Accuracy: 0.7236
Epoch 3/15, Loss: 0.7412, Accuracy: 0.7524
Epoch 4/15, Loss: 0.6913, Accuracy: 0.7686


# **BERT**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tqdm import tqdm
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Prepare the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
def tokenize_data(texts, tokenizer, max_len):
    return tokenizer(texts, padding=True, truncation=True, max_length=max_len, return_tensors="pt")

max_len = 256
train_encodings = tokenize_data(train_df['text'].tolist(), tokenizer, max_len)
test_encodings = tokenize_data(test_df['text'].tolist(), tokenizer, max_len)
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_df['label'])
y_test = label_encoder.transform(test_df['label'])

class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

train_dataset = TextDataset(train_encodings, y_train)
test_dataset = TextDataset(test_encodings, y_test)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

#pre-trained BERT
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_)).to(device)
optimizer = AdamW(model.parameters(), lr=2e-5)
epochs = 7
total_steps = len(train_loader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

def train_model(model, train_loader, optimizer, scheduler, device, epochs):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        correct_predictions = 0
        total_predictions = 0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            batch = {k: v.to(device) for k, v in batch.items()}

            optimizer.zero_grad()
            outputs = model(**batch)
            loss = outputs.loss
            logits = outputs.logits

            loss.backward()
            optimizer.step()
            scheduler.step()

            total_loss += loss.item()
            predictions = torch.argmax(logits, dim=-1)
            correct_predictions += (predictions == batch['labels']).sum().item()
            total_predictions += batch['labels'].size(0)

        avg_loss = total_loss / len(train_loader)
        accuracy = correct_predictions / total_predictions
        print(f"Epoch {epoch + 1}/{epochs} - Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

train_model(model, train_loader, optimizer, scheduler, device, epochs)
def evaluate_model(model, test_loader, device):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Evaluating"):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=-1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(batch['labels'].cpu().numpy())

    return all_preds, all_labels

y_test_pred, y_test_labels = evaluate_model(model, test_loader, device)

print("BERT Transformer Model Evaluation:")
print(f"Accuracy: {accuracy_score(y_test_labels, y_test_pred)}")
print(classification_report(y_test_labels, y_test_pred))



Using device: cuda


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1/7: 100%|██████████| 1635/1635 [19:34<00:00,  1.39it/s]


Epoch 1/7 - Loss: 0.7709, Accuracy: 0.7505


Epoch 2/7: 100%|██████████| 1635/1635 [19:35<00:00,  1.39it/s]


Epoch 2/7 - Loss: 0.4871, Accuracy: 0.8454


...

Epoch 6/7 - Loss: 0.0703, Accuracy: 0.9820


Epoch 7/7: 100%|██████████| 1635/1635 [19:32<00:00,  1.39it/s]


Epoch 7/7 - Loss: 0.0442, Accuracy: 0.9896


Evaluating: 100%|██████████| 409/409 [01:38<00:00,  4.14it/s]

BERT Transformer Model Evaluation:
Accuracy: 0.8135515448149281
              precision    recall  f1-score   support

           0       0.88      0.84      0.85       790
           1       0.75      0.79      0.77       807
           2       0.79      0.76      0.78       858
           3       0.76      0.72      0.74       826
           4       0.67      0.74      0.71       822
           5       0.99      0.97      0.98       751
           6       0.87      0.86      0.87       822
           7       0.83      0.84      0.84       862

    accuracy                           0.81      6538
   macro avg       0.82      0.82      0.82      6538
weighted avg       0.82      0.81      0.81      6538

